In [1]:
import json
import random
import os
import logging
import pickle
import string
import re
from pathlib import Path
from collections import Counter, OrderedDict, defaultdict as ddict
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset

In [23]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\salman\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [22]:
!pip install nltk

  Using cached nltk-3.5.zip (1.4 MB)
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434677 sha256=8b8acd13d68d8c5b1bb738294471d2f475bd6357247a3c2bf206e1813ad0a1ad
  Stored in directory: c:\users\salman\appdata\local\pip\cache\wheels\ff\d5\7b\f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
Successfully built nltk


In [2]:

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    data_dict = {'question': [], 'context': [], 'id': [], 'answer': []}
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in  qa['answers']:
                    data_dict['question'].append(question)
                    data_dict['context'].append(context)
                    data_dict['id'].append(qa['id'])
                    data_dict['answer'].append(answer)
    id_map = ddict(list)
    for idx, qid in enumerate(data_dict['id']):
        id_map[qid].append(idx)

    data_dict_collapsed = {'question': [], 'context': [], 'id': [], 'answer': []}
    for qid in id_map:
        ex_ids = id_map[qid]
        all_answers = [data_dict['answer'][idx] for idx in ex_ids]
        data_dict_collapsed['question'].append(data_dict['question'][ex_ids[0]])
        data_dict_collapsed['context'].append(data_dict['context'][ex_ids[0]])
        data_dict_collapsed['id'].append(qid)
        data_dict_collapsed['answer'].append({'answer_start': [answer['answer_start'] for answer in all_answers],
                                              'text': [answer['text'] for answer in all_answers]})
    return data_dict_collapsed


In [18]:
data_dict_collapsed = read_squad(r'D:\Jupyter\CS224N_2021_Winter_SCPD\robustqa-main\datasets_negative\indomain_train\squad')

In [19]:
data_dict_collapsed

{'question': ['What sits on top of the Main Building at Notre Dame?',
  'Which prize did Frederick Buechner create?',
  'In what year was the College of Engineering at Notre Dame formed?',
  'How many departments are within the Stinson-Remick Hall of Engineering?',
  'The College of Science began to offer civil engineering courses beginning at what time at Notre Dame?',
  'What entity provides help with the management of time for new students at Notre Dame?',
  'Which organization declared the First Year of Studies program at Notre Dame "outstanding?"',
  'What institute at Notre Dame studies  the reasons for violent conflict?',
  "What is the title of Notre Dame's Theodore Hesburgh?",
  'In what year was the Joan B. Kroc Institute for International Peace Studies founded?',
  'What company did Ray Kroc own?',
  'What is the daily student paper at Notre Dame called?',
  'How many student news papers are found at Notre Dame?',
  'What is the name of the main library at Notre Dame?',
  'I

In [ ]:
1. Seperate out questions, answers, 

In [20]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [65]:
wordnet.synsets('sits')

[Synset('sit.v.01'),
 Synset('sit.v.02'),
 Synset('sit_down.v.01'),
 Synset('sit.v.04'),
 Synset('model.v.03'),
 Synset('ride.v.01'),
 Synset('sit.v.07'),
 Synset('baby-sit.v.02'),
 Synset('seat.v.01'),
 Synset('sit.v.10')]

In [66]:
get_synonyms('sits')

['posture',
 'sit down',
 'baby sit',
 'sit',
 'sit around',
 'pose',
 'model',
 'seat',
 'ride']

In [53]:
aug_qs = []
for question in data_dict_collapsed['question'][:30]:
    new_q = ''
    for word in question.split(' '):
#         new_q = ''
        if len(get_synonyms(word)) == 0:
            a = word
        else:
            a = get_synonyms(word)[0]
        new_q += a + ' '
    aug_qs.append(new_q)

In [56]:
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            #print("replaced", random_word, "with", synonym)
            num_replaced += 1
        if num_replaced >= n: #only replace up to n words
            break

    #this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words

In [58]:
synonym_replacement(['college', 'what'], 2)

NameError: name 'stop_words' is not defined

In [55]:
data_dict_collapsed['question'][:30]

['What sits on top of the Main Building at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'In what year was the College of Engineering at Notre Dame formed?',
 'How many departments are within the Stinson-Remick Hall of Engineering?',
 'The College of Science began to offer civil engineering courses beginning at what time at Notre Dame?',
 'What entity provides help with the management of time for new students at Notre Dame?',
 'Which organization declared the First Year of Studies program at Notre Dame "outstanding?"',
 'What institute at Notre Dame studies  the reasons for violent conflict?',
 "What is the title of Notre Dame's Theodore Hesburgh?",
 'In what year was the Joan B. Kroc Institute for International Peace Studies founded?',
 'What company did Ray Kroc own?',
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'What is the name of the main library at Notre Dame?',
 'In what year was the Theodor

In [60]:
get_synonyms('student')

['bookman', 'scholarly person', 'pupil', 'scholar', 'educatee']

In [54]:
aug_qs

['What posture along top of the inning of the master construction astatine Notre Dame? ',
 'Which dirty money come frederick Buechner create? ',
 'inward what twelvemonth constitute the college of mastermind astatine Notre madam formed? ',
 'How many department constitute inside the Stinson-Remick dorm of Engineering? ',
 'The college of science start to proffer civic mastermind grade first astatine what prison term astatine Notre Dame? ',
 'What entity bring home the bacon helper with the direction of prison term for unexampled bookman astatine Notre Dame? ',
 'Which organisation hold the beginning year of study curriculum astatine Notre madam "outstanding?" ',
 'What bring astatine Notre madam study  the grounds for trigger happy conflict? ',
 "What constitute the form of address of Notre Dame's Theodore Hesburgh? ",
 'inward what twelvemonth constitute the Joan B. Kroc bring for outside peacefulness study founded? ',
 'What troupe come re Kroc own? ',
 'What constitute the everyday 

In [52]:
x

'What posture along top of the inning of the master construction astatine Notre Dame? '